# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
#pip install cartopy geoviews pyproj
import hvplot.pandas
import pandas as pd
import requests
 

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,boca do acre,-8.7522,-67.3978,30.32,66,23,1.87,BR,2025-03-24
1,1,upington,-28.4478,21.2561,27.21,57,50,1.54,ZA,2025-03-24
2,2,aasiaat,68.7098,-52.8699,-9.90,80,100,1.98,GL,2025-03-24
3,3,puerto quijarro,-17.7833,-57.7667,31.50,55,100,1.35,BO,2025-03-24
4,4,waitangi,-43.9535,-176.5597,14.90,98,7,1.34,NZ,2025-03-24


In [3]:
city_data_df.describe().loc[["min", "max"]].T

,min,max
City_ID,0.0000,600.0000
Lat,-54.8000,78.2186
Lng,-178.1667,177.4833
Max Temp,-31.2700,37.6100
Humidity,5.0000,100.0000
Cloudiness,0.0000,100.0000
Wind Speed,0.0000,14.4000


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 896,
    frame_height = 504,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5, 
)

# Display the map
hvplot.save(humidity_map, "output_data/humidity_map.html")
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

**Ideal Weather**: sweet spot where you can ski in the morning and hike in the afternoon (especially in the Pacific Northwest in late spring)
- A max temperature lower than 72 degrees but higher than 38 F ( 38-72 )
- Moderately humid (range: 40 to 65)
- Partly cloudy (range: 30 to 60)

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[
    (city_data_df['Max Temp'] <= 72)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()
countIdeal = len(cities_narrow_lats_df)
print(f"Number of cities with ideal weather: {countIdeal}")

# Display sample data
ideal_weather_city_df

Number of cities with ideal weather: 601


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,boca do acre,-8.7522,-67.3978,30.32,66,23,1.87,BR,2025-03-24
1,1,upington,-28.4478,21.2561,27.21,57,50,1.54,ZA,2025-03-24
2,2,aasiaat,68.7098,-52.8699,-9.90,80,100,1.98,GL,2025-03-24
3,3,puerto quijarro,-17.7833,-57.7667,31.50,55,100,1.35,BO,2025-03-24
4,4,waitangi,-43.9535,-176.5597,14.90,98,7,1.34,NZ,2025-03-24
...,...,...,...,...,...,...,...,...,...,...
596,596,sao vicente,-23.9631,-46.3919,28.92,51,17,2.34,BR,2025-03-24
597,597,macapa,0.0389,-51.0664,27.71,78,40,4.63,BR,2025-03-24
598,598,papao,-17.7500,-149.5500,29.82,80,65,10.26,PF,2025-03-24
599,599,rondonopolis,-16.4708,-54.6356,28.86,72,100,1.48,BR,2025-03-24


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

/var/folders/p7/44v4jhr91plb0p3405m1crv80000gn/T/ipykernel_30540/3885843272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
0,boca do acre,BR,-8.7522,-67.3978,66,
1,upington,ZA,-28.4478,21.2561,57,
2,aasiaat,GL,68.7098,-52.8699,80,
3,puerto quijarro,BO,-17.7833,-57.7667,55,
4,waitangi,NZ,-43.9535,-176.5597,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
limit = 7 
radius = 10000
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
boca do acre - nearest hotel: No hotel found
puerto quijarro - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Dali
west island - nearest hotel: Cocos Village Bungalows
san patricio - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
ati - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
bethel - nearest hotel: Hampton Inn Danbury
stephenville - nearest hotel: The Wyatt
tuskegee - nearest hotel: No hotel found
nuh - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
benjamin constant - nearest hotel: No hotel found
cockburn town - nearest hotel: The Salt Raker Inn
karema - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
flying fish cove - nearest hotel: Christmas Island Lodge
zouerate - nearest hotel: فندق تيرس زمور
soure - nearest hotel: Hotel Marajó
karratha - nearest hotel: Karratha International Hotel
utrik - near

,City,Country,Lat,Lng,Humidity,Hotel Name
0,boca do acre,BR,-8.7522,-67.3978,66,No hotel found
3,puerto quijarro,BO,-17.7833,-57.7667,55,No hotel found
5,constantia,RO,44.1833,28.6500,86,Hotel Dali
6,west island,CC,-12.1568,96.8225,94,Cocos Village Bungalows
7,san patricio,US,28.0170,-97.5169,68,No hotel found
...,...,...,...,...,...,...
595,vilhena,BR,-12.7406,-60.1458,89,No hotel found
596,sao vicente,BR,-23.9631,-46.3919,51,Pousada Vitória
597,macapa,BR,0.0389,-51.0664,78,Equatorial Hotel
598,papao,PF,-17.7500,-149.5500,80,Hiti Moana Villa Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hvplot.save(hotel_map, "output_data/hotel_map.html")
hotel_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)